In [1]:
import numpy as np
import tensorflow as tf

np.__version__, tf.__version__

('1.18.1', '2.1.0')

TensorFlow’s API revolves around **<font color='red'>tensors</font>**, which <font color='crimson'>flow from operation to operation</font> — hence the name TensorFlow.


A tensor is <font color='crimson'>very similar to a NumPy ndarray: it is usually a multidimensional array, but it can also hold a scalar</font> (a simple value, such as 42).


These tensors will be important when we create custom cost functions, custom metrics, custom layers, and more, so let’s see how to create and manipulate them.

# 1. Tensors and Operations

**Create a tensor with `tf.constant()`.**

In [2]:
# Create a tensor
tf.constant([[1., 2., 3.], [4, 5, 6]])  # matrix

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
# Create a tensor
# If dtype is specified the resulting tensor values are cast to the requested dtype.
tf.constant(42)  # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [32]:
tf.constant([1, 2, 3, 4, 5, 6], dtype=None, shape=None)

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([1, 2, 3, 4, 5, 6], dtype=int32)>

In [33]:
# If shape is set, the value is reshaped to match.
tf.constant([1, 2, 3, 4, 5, 6], dtype=None, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
array([[1, 2, 3],
       [4, 5, 6]], dtype=int32)>

In [35]:
# If shape is set, the value is reshaped to match.
# Scalars are expanded to fill the shape.
tf.constant(42., dtype=tf.float64, shape=(2, 3))

<tf.Tensor: shape=(2, 3), dtype=float64, numpy=
array([[42., 42., 42.],
       [42., 42., 42.]])>

Just like an ndarray, **a `tf.Tensor` has a shape and a data type (`dtype`):**

In [4]:
t = tf.constant([[1., 2., 3.], [4, 5, 6]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [5]:
t.shape

TensorShape([2, 3])

In [6]:
t.dtype

tf.float32

**Indexing works much like in NumPy:**

In [7]:
t[:, 1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [8]:
t[..., 1, tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [9]:
t[..., 1]

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([2., 5.], dtype=float32)>

**Most importantly, <font color='crimson'>all sorts of tensor operations are available:</font>**

In [10]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [11]:
tf.add(t, 10)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [12]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [13]:
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

- `t + 10` is equivalent to calling `tf.add(t, 10)` (indeed, Python calls the magic method `t.__add__(10)`, which just calls `tf.add(t, 10)`). Other operators like `-` and `*` are also supported.


- The `@` operator was added in Python 3.5, for matrix multiplication: it is equivalent to calling the `tf.matmul()` function.

You will find <font color='crimson'>all the basic math operations you need</font> (`tf.add()`, `tf.multiply()`, `tf.square()`, `tf.exp()`, `tf.sqrt()`, etc.) and **<font color='crimson'>most operations that you can find in NumPy</font> (e.g., `tf.reshape()`, `tf.squeeze()`, `tf.tile()`).

**<font color='crimson'>Some functions have a different name than in NumPy.</font>** For instance, `tf.reduce_mean()`, `tf.reduce_sum()`, `tf.reduce_max()`, and `tf.math.log()` are the equivalent of `np.mean()`, `np.sum()`, `np.max()` and `np.log()`.


**<font color='crimson'>When the name differs, there is often a good reason for it.</font>** For example,

- in TensorFlow you must write `tf.transpose(t)`; you cannot just write `t.T` like in NumPy. The reason is that the `tf.transpose()` function does not do exactly the same thing as NumPy’s `T` attribute: in TensorFlow, a new tensor is created with its own copy of the transposed data, while in NumPy, `t.T` is just a transposed view on the same data. 


- Similarly, the **`tf.reduce_sum()`** operation is **named this way because its GPU kernel (i.e., GPU implementation) uses a reduce algorithm that does not guarantee the order in which the elements are added: because 32-bit floats have limited precision, the result may change ever so slightly every time you call this operation.** The same is true of `tf.reduce_mean()` (but of course `tf.reduce_max()` is deterministic).

<div class="alert alert-block alert-info">
    <b><font color='crimson'>Many functions and classes have aliases.</font></b> For example, <code>tf.add()</code> and <code>tf.math.add()</code> are the same function.
    
This <b>allows TensorFlow to have concise (简洁的) names for the most common operations while preserving well-organized packages</b>.


A notable exception is <code>tf.math.log()</code>, which is commonly used but doesn’t have a <code>tf.log()</code> alias (as it might be confused with logging).
</div>

<div class="alert alert-block alert-success">
    <b><center>Keras's low-level API</center></b><br>
    The Keras API has its own low-level API, located in <code>keras.backend</code>. It includes functions like <code>square()</code>, <code>exp()</code>, and <code>sqrt()</code>.<br><br>
    In tf.keras, these functions (only cover a subset of all functions available in TensorFlow) generally just call the corresponding TensorFlow operations. If you want to write code that will be portable to other Keras implementations, you should use these Keras functions.
</div>

In [15]:
from tensorflow import keras

k = keras.backend

k.square(9) + 1

<tf.Tensor: shape=(), dtype=int32, numpy=82>

# 2. Tensors and NumPy

**<font color='cimson'>Tensors play nice with NumPy.</font>**

- <font color='cimson'>You can create a tensor from a NumPy array, and vice versa.</font>

In [16]:
a = np.array([2., 4, 5.])
a

array([2., 4., 5.])

In [17]:
a.dtype

dtype('float64')

In [18]:
# Create a tensor from ndarray
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [19]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [20]:
# Create a ndarray from tensor
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [21]:
# Create a ndarray from tensor
np.array(t)

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

- <font color='cimson'>You can even apply TensorFlow operations to NumPy arrays and NumPy operations to tensors.</font>

In [22]:
a

array([2., 4., 5.])

In [23]:
# Apply TF operations to ndarray
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [24]:
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [25]:
# Apply NumPy operations to tensor
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

<div class="alert alert-block alert-warning">
    <b>NumPy uses 64-bit precision by default, while TensorFlow uses 32-bit.</b><br><br>
    This is because 32-bit precision is generally more than enough for neural networks, plus it runs faster and uses less RAM.<br><br>
    So <b>when you create a tensor from a NumPy array, make sure to set dtype to tf.float32</b>.
</div>

# 3. Type conversions

**<font color='blue'>Type conversions can significantly hurt performance, and they can easily go unnoticed when they are done automatically.</font>**

To avoid this, **<font color='crimson'>TensorFlow does not perform any type conversions automatically: it just raises an exception if you try to execute an operation on tensors with incompatible types.</font>**

In [26]:
tf.constant(2.)

<tf.Tensor: shape=(), dtype=float32, numpy=2.0>

In [27]:
tf.constant(40)

<tf.Tensor: shape=(), dtype=int32, numpy=40>

In [28]:
tf.constant(2.) + tf.constant(40)  # incompatible types

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2] name: add/

In [29]:
tf.constant(2.) + tf.constant(40., dtype=tf.float64)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2] name: add/

虽然这个有点烦人，但是这是有充分理由的。

In [30]:
t2 = tf.constant(40., dtype=tf.float64)
t2

<tf.Tensor: shape=(), dtype=float64, numpy=40.0>

In [31]:
# Casts a tensor to a new type.
tf.cast(t2, dtype=tf.float32, name=None)

<tf.Tensor: shape=(), dtype=float32, numpy=40.0>

# 4. Variables

**The `tf.Tensor` values we’ve seen so far are immutable: you cannot modify them.**

This means that we cannot use regular tensors to implement weights in a neural network, since they need to be tweaked by backpropagation. Plus, other parameters may also need to change over time (e.g., a momentum optimizer keeps track of past gradients). What we need is a **`tf.Variable`**:

In [41]:
# Create a variable
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

**A `tf.Variable` acts much like a `tf.Tensor`**

- perform the sampe ops with it


- plays nicely with NumPy as well


- it's just picky (挑剔的) with types

<br>

**<font color='crimson'>It can also modified in place</font>** using the **`assign()`** method (or **`assign_add()`** or **`asssign_sub()`**, which increment or decrement the variable by the given value).


**<font color='crimson'>You can also modify individual cells (or slices)</font>**, by using the **cell’s (or slice’s) `assign()` method** (direct item assignment will not work) or by using the **`scatter_update()`** or **`scatter_nd_update()`** methods.

In [42]:
# Assigns a new value to the variable.
v.assign(2 * v)  # inplace

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [43]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [44]:
v[0, 1].assign(42)  # inplace

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [45]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [47]:
v[:, 0].assign([100., 100.])  # inplace

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [100.,  10.,  12.]], dtype=float32)>

In [48]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [100.,  10.,  12.]], dtype=float32)>

In [49]:
# Adds a value to this variable.
v + 1  # inplace=Fale

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[101.,  43.,   7.],
       [101.,  11.,  13.]], dtype=float32)>

In [50]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   6.],
       [100.,  10.,  12.]], dtype=float32)>

In [51]:
# Applies sparse assignment to individual values or slices in a Variable.
v.scatter_nd_update(
    # The indices to be used in the operation.
    indices=[[0, 0], [1, 2]],
    # The values to be used in the operation.
    updates=[10000., 10000.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[1.0e+04, 4.2e+01, 6.0e+00],
       [1.0e+02, 1.0e+01, 1.0e+04]], dtype=float32)>

In [52]:
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1.0e+04, 4.2e+01, 6.0e+00],
       [1.0e+02, 1.0e+01, 1.0e+04]], dtype=float32)>

<div class="alert alert-block alert-info">
    <b>In practice you will rarely have to create variables manually</b>, since Keras provides an <code>add_weight()</code> method that will take care of it for you, as we will see.<br><br>
    Moreover, <b>model parameters will generally be updated directly by the optimizers</b>, so you will rarely need to update variables manually.
</div>

# 5. Other data structures (TODO)

TODO...